Instructions
 1. Process the Text
 - Convert to Lowercase
 - Remove Punctuation & Numbers
 - Tokenization (Splitting text into individual words or tokens)
 - Removing Stop Words (e.g. "and", "the", "of")
 - Stemming/Lemmatization (reducing words to their base/root form) *optional

 2. Vectorize Text
 - Convert text into numerical format using TF-IDF or embeddings
 - TF-IDF weights the words based on their frequency in a document against their frequency across all documents
 - Word embeddings using Word2Vec, CloVe, or FastText can be used to convert words into vectors (better)

 3. Dimensionality Reduction with t-SNE
 - Choosing parameters (starting point 5 - 50, learning rate 10-1000)
 - Scaling - Might need to scae feature vectors before applying (MinMAxScaler or StandardScaler)
 - Applying t-SNE to reduce dimensionality of vectorized hymns (from scikit-learn)

 4. Visualization
 - Plotting (Matplotlib) to create a scatter plot of 2D points
 - Enhancements to make visualization more informative (optional?)

In [ ]:
from sklearn.feature_extraction.text import TfidVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
# Import hymns
hymns = []

def get_hymns(): 
    h = input("Enter the hymn number (1-525). Press enter if finished. ")
    if h:
        hymns.append(int(h))
        get_hymns()
    else:
        return hymns
    
get_hymns()

In [ ]:
# 1. Process the Text
